In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

!pip install bayesian-optimization

from sklearn.model_selection import KFold,StratifiedKFold   # K-fold CV 
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

import warnings
warnings.filterwarnings(action = 'ignore')

from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/train_df_noNAN.csv')
sub = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/submission.csv')

In [ ]:
train_df=data.copy()

In [ ]:
train_df.isna().sum()

REG_YYMM        0
CARD_SIDO_NM    0
CARD_CCG_NM     0
STD_CLSS_NM     0
HOM_SIDO_NM     0
HOM_CCG_NM      0
AGE             0
SEX_CTGO_CD     0
FLC             0
CSTMR_CNT       0
AMT             0
CNT             0
dtype: int64

In [ ]:
# train과 test셋을 만들 dataframe만들기
df_form=train_df.groupby(['CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM','HOM_SIDO_NM','HOM_CCG_NM','AGE','SEX_CTGO_CD','FLC'])['AMT'].sum().reset_index().drop('AMT',axis=1)

In [ ]:
df_form.head()

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,2,1
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3


In [ ]:
# 날짜 삽입
df_form['REG_YYMM'] =202001
df_202001= df_form.copy()
df_form['REG_YYMM'] =202002
df_202002= df_form.copy()
df_form['REG_YYMM'] =202003
df_202003= df_form.copy()
df_form['REG_YYMM'] =202004
df_202004= df_form.copy()
df_form['REG_YYMM'] =202007
df_202007= df_form.copy()

df_2020 = pd.concat([df_202001,df_202002,df_202003,df_202004,df_202007])
df_2020 = df_2020[['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','HOM_CCG_NM', 'AGE', 'SEX_CTGO_CD', 'FLC']]
df_2020.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC
0,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1
1,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,2,1
2,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2
3,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2
4,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3


In [ ]:
df_train= df_2020[df_2020['REG_YYMM']<202004].copy()
df_test= df_2020[df_2020['REG_YYMM']>=202004].copy()

---

# 파생변수 만들기

전 월의 매출액 삽입
- 4월 다음 바로 7월을 예측해야하기 때문에 당 월의 전 월을 넣는 것은 바람직하지 않음.
- 3달,6달,9달,12달 전의 AMT를 넣도록 하자. (이 방식대로 하면 CNT와 CSTMR_CNT도 고려할 수가 있음)

In [ ]:
# 기준에 따른 전 월 AMT_sum 
CARD_AMT = data.groupby(['REG_YYMM','CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM'])['AMT'].sum().reset_index(name='previous_CARD_AMT')
HOM_AMT = data.groupby(['REG_YYMM','HOM_SIDO_NM','HOM_CCG_NM','STD_CLSS_NM'])['AMT'].sum().reset_index(name='previous_HOM_AMT')
ASF_AMT = data.groupby(['REG_YYMM','AGE','SEX_CTGO_CD','FLC'])['AMT'].sum().reset_index(name='previous_ASF_AMT')

# 기준에 따른 전 월 CNT_sum
CARD_CNT = data.groupby(['REG_YYMM','CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM'])['CNT'].sum().reset_index(name='previous_CARD_CNT')
HOM_CNT = data.groupby(['REG_YYMM','HOM_SIDO_NM','HOM_CCG_NM','STD_CLSS_NM'])['CNT'].sum().reset_index(name='previous_HOM_CNT')
ASF_CNT = data.groupby(['REG_YYMM','AGE','SEX_CTGO_CD','FLC'])['CNT'].sum().reset_index(name='previous_ASF_CNT')

# 기준에 따른 전 월 CSTMR_CNT_sum
CARD_CSTMR_CNT = data.groupby(['REG_YYMM','CARD_SIDO_NM','CARD_CCG_NM','STD_CLSS_NM'])['CSTMR_CNT'].sum().reset_index(name='previous_CARD_CSTMR_CNT')
HOM_CSTMR_CNT = data.groupby(['REG_YYMM','HOM_SIDO_NM','HOM_CCG_NM','STD_CLSS_NM'])['CSTMR_CNT'].sum().reset_index(name='previous_HOM_CSTMR_CNT')
ASF_CSTMR_CNT = data.groupby(['REG_YYMM','AGE','SEX_CTGO_CD','FLC'])['CSTMR_CNT'].sum().reset_index(name='previous_ASF_CSTMR_CNT')


# '날짜' 변수를 3,6,9,12씩 더해주기
def plus_reg(df,what,AMTorCNT_df,num):
  #원본 날짜 저장
  original_reg = AMTorCNT_df['REG_YYMM'].copy()

  AMTorCNT_df['REG_YYMM'] = AMTorCNT_df['REG_YYMM'].apply(lambda x: x+num)
  AMTorCNT_df['REG_YYMM'] = AMTorCNT_df['REG_YYMM'].apply(lambda x: x + 100 -12 if x > 201912 and x < 202000 else x)

  if what == 'train':
    merge_df = AMTorCNT_df[(AMTorCNT_df['REG_YYMM']==202001)|(AMTorCNT_df['REG_YYMM']==202002)|(AMTorCNT_df['REG_YYMM']==202003)]
  else:
    merge_df = AMTorCNT_df[(AMTorCNT_df['REG_YYMM']==202004)|(AMTorCNT_df['REG_YYMM']==202007)]

  df = df.merge(merge_df,on=[col for col in merge_df.columns[:-1]],how='outer')
  df.rename(columns={df.columns[-1]:df.columns[-1]+'_%s'%(num)},inplace=True)
  AMTorCNT_df['REG_YYMM'] = original_reg

  return df

In [ ]:
for AMT in [CARD_AMT,HOM_AMT,ASF_AMT,CARD_CNT,HOM_CNT,ASF_CNT,CARD_CSTMR_CNT,HOM_CSTMR_CNT,ASF_CSTMR_CNT]:
  for num in [3,6,9,12]:
    df_train=plus_reg(df_train,'train',AMT,num)

df_train.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,previous_CARD_AMT_3,previous_CARD_AMT_6,previous_CARD_AMT_9,previous_CARD_AMT_12,previous_HOM_AMT_3,previous_HOM_AMT_6,previous_HOM_AMT_9,previous_HOM_AMT_12,previous_ASF_AMT_3,previous_ASF_AMT_6,previous_ASF_AMT_9,previous_ASF_AMT_12,previous_CARD_CNT_3,previous_CARD_CNT_6,previous_CARD_CNT_9,previous_CARD_CNT_12,previous_HOM_CNT_3,previous_HOM_CNT_6,previous_HOM_CNT_9,previous_HOM_CNT_12,previous_ASF_CNT_3,previous_ASF_CNT_6,previous_ASF_CNT_9,previous_ASF_CNT_12,previous_CARD_CSTMR_CNT_3,previous_CARD_CSTMR_CNT_6,previous_CARD_CSTMR_CNT_9,previous_CARD_CSTMR_CNT_12,previous_HOM_CSTMR_CNT_3,previous_HOM_CSTMR_CNT_6,previous_HOM_CSTMR_CNT_9,previous_HOM_CSTMR_CNT_12,previous_ASF_CSTMR_CNT_3,previous_ASF_CSTMR_CNT_6,previous_ASF_CSTMR_CNT_9,previous_ASF_CSTMR_CNT_12
0,202001,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,17370919.0,13430550.0,16889940.0,24027180.0,19686719.0,13170200.0,18203540.0,25283480.0,255306663988,256512996459,222780321814,208963599954,144.0,117.0,112.0,150.0,184.0,117.0,131.0,160.0,26541802,25451347,23083184,19572750,133.0,108.0,113.0,142.0,163.0,110.0,132.0,149.0,10071849,10398235,8983445,8147930
1,202001,경기,성남시 분당구,건강보조식품 소매업,강원,강릉시,20s,1,1,75345518.0,69396341.0,67998415.0,76023039.0,19686719.0,13170200.0,18203540.0,25283480.0,255306663988,256512996459,222780321814,208963599954,992.0,851.0,858.0,856.0,184.0,117.0,131.0,160.0,26541802,25451347,23083184,19572750,879.0,772.0,770.0,771.0,163.0,110.0,132.0,149.0,10071849,10398235,8983445,8147930
2,202001,서울,용산구,건강보조식품 소매업,강원,강릉시,20s,1,1,11383354.0,14255780.0,13616003.0,16785786.0,19686719.0,13170200.0,18203540.0,25283480.0,255306663988,256512996459,222780321814,208963599954,302.0,549.0,841.0,607.0,184.0,117.0,131.0,160.0,26541802,25451347,23083184,19572750,274.0,443.0,700.0,543.0,163.0,110.0,132.0,149.0,10071849,10398235,8983445,8147930
3,202001,강원,동해시,건강보조식품 소매업,강원,동해시,20s,1,1,12965900.0,7897250.0,9326333.0,11266370.0,15138900.0,8483140.0,9326333.0,11464070.0,255306663988,256512996459,222780321814,208963599954,61.0,33.0,52.0,66.0,86.0,49.0,52.0,71.0,26541802,25451347,23083184,19572750,54.0,33.0,61.0,72.0,77.0,46.0,61.0,76.0,10071849,10398235,8983445,8147930
4,202001,강원,원주시,건강보조식품 소매업,강원,원주시,20s,1,1,35104188.0,22708178.0,29921685.0,45762905.0,43421578.0,27693178.0,35665855.0,61265065.0,255306663988,256512996459,222780321814,208963599954,253.0,233.0,287.0,324.0,421.0,303.0,362.0,454.0,26541802,25451347,23083184,19572750,238.0,232.0,278.0,312.0,356.0,277.0,325.0,389.0,10071849,10398235,8983445,8147930


In [ ]:
for AMT in [CARD_AMT,HOM_AMT,ASF_AMT,CARD_CNT,HOM_CNT,ASF_CNT,CARD_CSTMR_CNT,HOM_CSTMR_CNT,ASF_CSTMR_CNT]:
  for num in [3,6,9,12]:
    df_test=plus_reg(df_test,'test',AMT,num)

df_test.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,previous_CARD_AMT_3,previous_CARD_AMT_6,previous_CARD_AMT_9,previous_CARD_AMT_12,previous_HOM_AMT_3,previous_HOM_AMT_6,previous_HOM_AMT_9,previous_HOM_AMT_12,previous_ASF_AMT_3,previous_ASF_AMT_6,previous_ASF_AMT_9,previous_ASF_AMT_12,previous_CARD_CNT_3,previous_CARD_CNT_6,previous_CARD_CNT_9,previous_CARD_CNT_12,previous_HOM_CNT_3,previous_HOM_CNT_6,previous_HOM_CNT_9,previous_HOM_CNT_12,previous_ASF_CNT_3,previous_ASF_CNT_6,previous_ASF_CNT_9,previous_ASF_CNT_12,previous_CARD_CSTMR_CNT_3,previous_CARD_CSTMR_CNT_6,previous_CARD_CSTMR_CNT_9,previous_CARD_CSTMR_CNT_12,previous_HOM_CSTMR_CNT_3,previous_HOM_CSTMR_CNT_6,previous_HOM_CSTMR_CNT_9,previous_HOM_CSTMR_CNT_12,previous_ASF_CSTMR_CNT_3,previous_ASF_CSTMR_CNT_6,previous_ASF_CSTMR_CNT_9,previous_ASF_CSTMR_CNT_12
0,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,26370210.0,17370919.0,13430550.0,16889940.0,27945010.0,19686719.0,13170200.0,18203540.0,2.531744e+11,255306663988,256512996459,222780321814,174.0,144.0,117.0,112.0,204.0,184.0,117.0,131.0,22424721.0,26541802,25451347,23083184,174.0,133.0,108.0,113.0,197.0,163.0,110.0,132.0,9583333.0,10071849,10398235,8983445
1,202004,경기,성남시 분당구,건강보조식품 소매업,강원,강릉시,20s,1,1,97624466.0,75345518.0,69396341.0,67998415.0,27945010.0,19686719.0,13170200.0,18203540.0,2.531744e+11,255306663988,256512996459,222780321814,923.0,992.0,851.0,858.0,204.0,184.0,117.0,131.0,22424721.0,26541802,25451347,23083184,826.0,879.0,772.0,770.0,197.0,163.0,110.0,132.0,9583333.0,10071849,10398235,8983445
2,202004,서울,용산구,건강보조식품 소매업,강원,강릉시,20s,1,1,23417687.0,11383354.0,14255780.0,13616003.0,27945010.0,19686719.0,13170200.0,18203540.0,2.531744e+11,255306663988,256512996459,222780321814,477.0,302.0,549.0,841.0,204.0,184.0,117.0,131.0,22424721.0,26541802,25451347,23083184,431.0,274.0,443.0,700.0,197.0,163.0,110.0,132.0,9583333.0,10071849,10398235,8983445
3,202004,강원,동해시,건강보조식품 소매업,강원,동해시,20s,1,1,16798820.0,12965900.0,7897250.0,9326333.0,16798820.0,15138900.0,8483140.0,9326333.0,2.531744e+11,255306663988,256512996459,222780321814,85.0,61.0,33.0,52.0,85.0,86.0,49.0,52.0,22424721.0,26541802,25451347,23083184,86.0,54.0,33.0,61.0,86.0,77.0,46.0,61.0,9583333.0,10071849,10398235,8983445
4,202004,강원,원주시,건강보조식품 소매업,강원,원주시,20s,1,1,47346738.0,35104188.0,22708178.0,29921685.0,52772318.0,43421578.0,27693178.0,35665855.0,2.531744e+11,255306663988,256512996459,222780321814,402.0,253.0,233.0,287.0,448.0,421.0,303.0,362.0,22424721.0,26541802,25451347,23083184,387.0,238.0,232.0,278.0,424.0,356.0,277.0,325.0,9583333.0,10071849,10398235,8983445


In [ ]:
df_train.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/df_train.csv', encoding='utf-8-sig',index=False)
df_test.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/df_test.csv', encoding='utf-8-sig',index=False)